In [22]:
import matplotlib.pyplot as plt
import numpy as np 
import seaborn as sns
import statsmodels.api as sm
import pandas as pd 
data = pd.read_csv('nesarc_pds.csv')

In [3]:
df = data[['IDNUM', 'MAJORDEP12', 'NMANDX12', 'NHYPO12DX', 'GENAXDX12','PANDXP12', 'APANDX12', 'SOCPDX12', 'ANTISOCDX2', 'AVOIDPDX2', 'OBCOMDX2', 'PARADX2', 'SCHIZDX2', 'HISTDX2', 'ALCABDEP12DX', 'TAB12MDX', 'STIM12ABDEP', 'PAN12ABDEP', 'SED12ABDEP', 'TRAN12ABDEP', 'COC12ABDEP', 'SOL12ABDEP', 'MAR12ABDEP', 'HER12ABDEP']]
df = df.rename(columns={'NMANDX12':'MANIC12', 'NHYPO12DX':'HYPOMANIC12', 'GENAXDX12':'GAD12','SOCPDX12' : 'SOCIAL12', 'ANTISOCDX2':'ANTISOC', 'AVOIDPDX2':'AVOIDANT', 'OBCOMDX2':'OCPD', 'PARADX2':'PARANOID', 'SCHIZDX2':'SCHIZOID', 'HISTDX2':'HISTRI', 'ALCABDEP12DX':'ALCOHOL12', 'TAB12MDX':'NICOTINE12', 'STIM12ABDEP':'AMPH12', 'PAN12ABDEP':'OPI12', 'SED12ABDEP': 'SED12', 'TRAN12ABDEP': 'TRANQUI12', 'COC12ABDEP': 'COCAINE12', 'SOL12ABDEP': 'INHALANT12', 'MAR12ABDEP': 'CANNABIS12', 'HER12ABDEP': 'HEROIN12', 'PANDXP12': 'PAN12','APANDX12': 'AGORA_PAN12' })
affective_disorders = ['MAJORDEP12', 'HYPOMANIC12', 'MANIC12']
anxiety_disorders = ['GAD12', 'PAN12', 'AGORA_PAN12']
personality_disorders = ['SOCIAL12', 'ANTISOC', 'AVOIDANT', 'OCPD', 'PARANOID', 'SCHIZOID', 'HISTRI']
df['affective'] = df[affective_disorders].any(axis=1).astype(int)
df['personality'] = df[personality_disorders].any(axis=1).astype(int)
df['anxiety'] = df[anxiety_disorders].any(axis=1).astype(int)
covariates = ['AGE', 'S1Q7A1', 'S1Q7A6', 'SEX', 'S1Q1C', 'S1F1C', 'S1Q1D1', 'S1Q1D2', 'S1Q1D3', 'S1Q1D4', 'S1Q1D5', 'S1F1D', 'MARITAL', 'S1Q11A', 'S1Q10A']


In [4]:
drugs_column = ['ALCOHOL12', 'NICOTINE12', 'AMPH12', 'OPI12',
       'SED12', 'TRANQUI12', 'COCAINE12', 'INHALANT12', 'CANNABIS12',
       'HEROIN12']
disorders = ['Affective Disorder', 'Personality Disorder', 'Anxiety Disorder']
df[drugs_column] = df[drugs_column].replace({2: 1, 3: 1})

In [ ]:
stimulants = ['AMPH12', 'NICOTINE12', 'COCAINE12']
depressants = ['ALCOHOL12', 'TRANQUI12', 'SED12']
analgesics = ['OPI12', 'HEROIN12', 'CANNABIS12']
pd.set_option('display.max_columns', None)
df['stimulant'] = df[stimulants].any(axis=1).astype(int)
df['depressant'] = df[depressants].any(axis=1).astype(int)
df['analgesic'] = df[analgesics].any(axis=1).astype(int)
stimulant_counts = df['stimulant'].value_counts()
stimulant_counts = stimulant_counts.sort_index()
print(stimulant_counts)

In [ ]:
covariatelist = ['AGE', 'S1Q7A1', 'S1Q7A6', 'SEX', 'S1Q1C', 'S1F1C', 'S1Q1D1', 'S1Q1D2', 'S1Q1D3', 'S1Q1D4', 'S1Q1D5', 'S1F1D', 'MARITAL', 'S1Q11A', 'S1Q10A']
df = df.join(data[covariatelist])
print(df.head())

In [7]:
# recode binary variables from 1/2 to 0/1
binary_vars = ['S1Q7A1', 'S1Q7A6', 'SEX', 'S1Q1C', 'S1F1C', 'S1Q1D1', 'S1Q1D2', 'S1Q1D3', 'S1Q1D4', 'S1Q1D5', 'S1F1D']
for var in binary_vars:
    df[var] = df[var].replace({2:0})
#dummy var for marital
df['MARITAL_BINARY'] = df['MARITAL'].apply(lambda x: 0 if x == 1 else 1)


covariatelist = ['AGE', 'S1Q7A1', 'S1Q7A6', 'SEX', 'S1Q1C', 'S1F1C', 'S1Q1D1', 'S1Q1D2', 'S1Q1D3', 'S1Q1D4', 'S1Q1D5', 'S1F1D', 'MARITAL_BINARY', 'S1Q11A', 'S1Q10A']

# standardizing continuous variables
continuous_vars = ['AGE', 'S1Q11A', 'S1Q10A']
for var in continuous_vars:
    df[f'{var}_std'] = (df[var] - df[var].mean()) / df[var].std()


In [ ]:
print(df['personality'].value_counts())
print(df['affective'].value_counts())
print(df['anxiety'].value_counts())

#### Collinearity check

In [ ]:
correlation_matrix = df[['affective', 'personality', 'anxiety']].corr()

print(correlation_matrix)

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

# calculate VIF for each variable
def calculate_vif(df, features):
    X = df[features] #dataframe with only specified fatures
    X['Intercept'] = 1  

    vif_data = pd.DataFrame()
    vif_data['Variable'] = X.columns
    vif_data['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])] #iterating through column indices in X, passes data vals + column index to VIF fxn 
    
    return vif_data.drop(index=[len(vif_data)-1])   #excludes intercept since it's not a feature

# checking multi collinearity for main predictors
features = ['affective', 'personality', 'anxiety']
vif_values = calculate_vif(df, features)

print(vif_values)


In [ ]:
stimulants = ['AMPH12', 'NICOTINE12', 'COCAINE12']
depressants = ['ALCOHOL12', 'TRANQUI12', 'SED12']
analgesics = ['OPI12', 'HEROIN12', 'CANNABIS12']
pd.set_option('display.max_columns', None)
df['stimulant'] = df[stimulants].any(axis=1).astype(int)
df['depressant'] = df[depressants].any(axis=1).astype(int)
df['analgesic'] = df[analgesics].any(axis=1).astype(int)
stimulant_counts = df['stimulant'].value_counts()
stimulant_counts = stimulant_counts.sort_index()
print(stimulant_counts)

In [ ]:
from scipy.stats import chi2_contingency

pairs = [
    ('affective', 'stimulant'),
    ('affective', 'depressant'),
    ('affective', 'analgesic'),
    ('anxiety', 'stimulant'),
    ('anxiety', 'analgesic'),
    ('anxiety', 'depressant'),
    ('personality', 'analgesic'),
    ('personality', 'depressant'),
    ('personality', 'stimulant')
]

# Storing results
results = []

for x, y in pairs:
    contingency_table = pd.crosstab(df[x], df[y])
    
    chi2, p_value, dof, expected = chi2_contingency(contingency_table)
    

    results.append((x, y, chi2, p_value, dof))

print(f"{'Variable 1':<15} {'Variable 2':<15} {'Chi-Square':<12} {'P-Value':<10} {'DF':<5}")
for x, y, chi2, p_value, dof in results:
    print(f"{x:<15} {y:<15} {chi2:<12.3f} {p_value:<10.3e} {dof:<5}")

# <u> Frequency Breakdown of Co-occurring Psychiatric Disorder and Addiction by Drug Class

In [ ]:
alpha = 0.05
n_comparisons = 9  # Total number of comparisons

adjusted_alpha = alpha / n_comparisons
print(f"Adjusted alpha: {adjusted_alpha:.5f}")
results = [
    ('affective', 'stimulant', 1205.116, 4.715e-264),
    ('affective', 'depressant', 552.168, 4.250e-122),
    ('affective', 'analgesic', 434.986, 1.335e-96),
    ('anxiety', 'stimulant', 681.129, 3.797e-150),
    ('anxiety', 'analgesic', 66.806, 2.995e-16),
    ('anxiety', 'depressant', 89.734, 2.724e-21),
    ('personality', 'analgesic', 504.492, 1.001e-111),
    ('personality', 'depressant', 613.321, 2.121e-135),
    ('personality', 'stimulant', 1309.068, 1.209e-286)
]

results_df = pd.DataFrame(results, columns=['Variable 1', 'Variable 2', 'Chi-Square', 'P-Value'])

# applying Bonferroni adjustment to p-values
results_df['Adjusted P-Value'] = results_df['P-Value'] * n_comparisons
results_df['Adjusted P-Value'] = results_df['Adjusted P-Value'].apply(lambda p: min(p, 1))  # Cap at 1

results_df['Significant'] = results_df['Adjusted P-Value'] < alpha

print(results_df)

In [ ]:
df['Affective Disorder'] = df[affective_disorders].any(axis=1).astype(int)
df['Personality Disorder'] = df[personality_disorders].any(axis=1).astype(int)
df['Anxiety Disorder'] = df[anxiety_disorders].any(axis=1).astype(int)
disorders = ['Affective Disorder', 'Personality Disorder', 'Anxiety Disorder']
df['Disorders'] = df[disorders].any(axis=1).astype(int)
drug_type = ['stimulant', 'depressant', 'analgesic']
dis_dtype_counts = pd.DataFrame(index=disorders, columns=drug_type)
for drug in drug_type:
    for disorder in disorders:
        dis_dtype_counts.loc[disorder, drug] = df[df[drug] == 1][disorder].sum()
colors = ['skyblue', '#ff9999', '#c2c2f0']
fig, ax = plt.subplots(figsize=(10, 7))
dis_dtype_counts.plot(kind='bar', stacked=True, color=colors, ax=ax)
totals = dis_dtype_counts.sum(axis=0)

for p in ax.patches:
    width, height = p.get_width(), p.get_height()
    x, y = p.get_xy()
    if height > 0:  # if there's height to bar, add text
        # Get the total for this specific disorder (column)
        column_idx = int(x)
        total = totals[column_idx]
        # Calculate percentage
        percentage = (height / total) * 100
        # Add percentage text
        ax.text(x + width/2, y + height/2, f'{percentage:.1f}%', 
                ha='center', va='center', fontsize=8, color='black')


plt.title('Comorbidity of Psychiatric Disorders and Drug Abuse by Pharmacological Class')
plt.xlabel('Psychiatric Disorders', labelpad=10)
plt.ylabel('Count of Drug Abuse Cases')
plt.xticks(rotation=0)
plt.legend(title='Drug Type')
plt.tight_layout()
plt.show()
other_stims = ['AMPH12', 'COCAINE12']
df['Stims Excluding Nic'] = df[other_stims].any(axis=1).astype(int)
drug_type2 = ['Stims Excluding Nic', 'NICOTINE12', 'analgesic', 'depressant']
dis_dtype_counts2 = pd.DataFrame(index=disorders, columns=drug_type2)

for drug in drug_type2:
    for disorder in disorders:
        dis_dtype_counts2.loc[disorder, drug] = df[df[drug] == 1][disorder].sum()
for disorder in disorders:
    for drug in drug_type2:
        # cases where both disorder and drug use are present, 'len' counts number of true cacses that match both conditionals 
        dis_dtype_counts2.loc[disorder, drug] = len(df[(df[disorder] == 1) & (df[drug] == 1)])

# summing rows horizontally, then dividng each value by row total
dis_dtype_pcts2 = dis_dtype_counts2.div(dis_dtype_counts2.sum(axis=1), axis=0) * 100

fig, ax = plt.subplots(figsize=(12, 6))

#tracks bottom of each segment so they stack instea of overlap
bottom = np.zeros(len(disorders))
colors = sns.color_palette('pastel', n_colors=len(drug_type2))

#each segment = drugype
for drug,color in zip(drug_type2, colors):
    values = dis_dtype_pcts2[drug] #list of percentages for drug type across disordrr
    plt.bar(disorders, values, bottom=bottom, label=drug, color = color)
    
    # i = index of values, v = percentage at that index
    for i, v in enumerate(values):
        if v > 0:  #  add label if segment is visible
            middle = bottom[i] + v/2 #calculating middle position
            plt.text(i, middle, f'{v:.1f}%', #formatting num to 1 decimal place w/ '%' symbol. 
                    ha='center', va='center')
    
    bottom += values

plt.title('Comorbidity of Psychiatric Disorders and Drug Abuse by Pharmacological Class (Isolating Nicotine)')
subtitle_text = 'Distinguishes Nicotine Addiction from Other Stimulant Drug of Abuse.'
plt.text(0.5, 0.97, subtitle_text, fontsize=10, color='gray', ha='center', va='bottom', transform=ax.transAxes)
plt.xlabel('Psychiatric Disorders', labelpad=10)
plt.ylabel('Percentage')
plt.legend(title='Drug Types', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

# Separate Logistic Regressions : Understanding the Relationship Between Specific Drug of Abuse Types and Psychiatric Disorders

In [34]:
import statsmodels.api as sm


## <u>Depressants<u>

In [ ]:
import statsmodels.api as sm
import numpy as np
X = df[['anxiety', 'affective', 'personality'] + covariatelist]
X = sm.add_constant(X)
y = df['depressant'] #dependent
model = sm.Logit(y, X)
result = model.fit()

print("\nLogistic Regression Results:")
print(result.summary())
np.exp(result.params)

params = result.params #exponentiating log odds to get odd ratios 
conf = result.conf_int() # confidence intervals 
conf['Odds Ratio'] = params
conf.columns = ['5%', '95%', 'Odds Ratio']
print(np.exp(conf))

### Visualization of Odds Ratio and Regression Coefficients w/ Depressants

In [ ]:
import matplotlib.pyplot as plt
odds_depress = {
    'Predictor': ['anxiety', 'affective', 'personality'],
    'Odds Ratio': [1.285288, 1.936388, 1.801178],
    '5% CI': [1.116501, 1.739744, 1.647060],
    '95% CI': [1.479592, 2.155258, 1.969717]
}

res_dep_df = pd.DataFrame(odds_depress)

plt.figure(figsize=(8, 6))
bars = plt.bar(res_dep_df['Predictor'], res_dep_df['Odds Ratio'], color=['hotpink', 'purple', 'orange'], edgecolor='black', alpha=0.7)

# confidence interval error bars; yerr = error bar length for datapoint i n y-axis direc.
plt.errorbar(res_dep_df['Predictor'], res_dep_df['Odds Ratio'], 
             yerr=[res_dep_df['Odds Ratio'] - res_dep_df['5% CI'], res_dep_df['95% CI'] - res_dep_df['Odds Ratio']], 
             fmt='o', color='black', capsize=5, alpha=0.9)

plt.axhline(y=1, color='gray', linestyle='--', linewidth=0.8, label='Baseline (OR=1)')
plt.title('Odds Ratios for Depressants', fontsize=14, weight = 'bold')
plt.ylabel('Odds Ratio', fontsize=12)
plt.xlabel('Predictor', fontsize=12)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)

for i, row in res_dep_df.iterrows():
    if row['5% CI'] > 1 or row['95% CI'] < 1:  #  if CI does not include 1, include asterickks. 
        plt.text(i, row['Odds Ratio'], '*', ha='center', va='bottom', fontsize=14, color='black')

plt.legend(loc='upper right', fontsize=10)

plt.tight_layout()
plt.show()

dep_coefficients = {
    'Predictors': ['anxiety', 'affective', 'personality'],
    'Coefficients': [ 0.2510, 0.6608, 0.5884],
    'Lower CI': [ 0.110, 0.554, 0.499],
    'Upper CI': [ 0.392, 0.768, 0.678]
}
depress_coefficients = pd.DataFrame(dep_coefficients)

colors = ['hotpink','teal', 'purple']

plt.figure(figsize=(8, 6))
bars = plt.bar(depress_coefficients['Predictors'], depress_coefficients['Coefficients'], color=colors, edgecolor='black', alpha=0.8)
plt.errorbar(
    depress_coefficients['Predictors'],
    depress_coefficients['Coefficients'],
    yerr=[depress_coefficients['Coefficients'] - depress_coefficients['Lower CI'], depress_coefficients['Upper CI'] - depress_coefficients['Coefficients']], #yerr = error bar length for datapoint i n y-axis direc.
    fmt='none',
    ecolor='black',
    capsize=5
)

for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height + 0.05,
             '*',
             ha='center', va='bottom', fontsize=14)
    
plt.title('Logistic Regression Coefficients for Depressants', fontsize=14, weight='bold')
plt.xlabel('Predictors', fontsize=12, weight='bold')
plt.ylabel('Coefficients', fontsize=12, weight='bold')
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()

plt.show()





## <u>Stimulants<u>

In [ ]:
X = df[['anxiety', 'affective', 'personality'] + covariatelist]
X = sm.add_constant(X)
y = df['stimulant']
model = sm.Logit(y, X)
result = model.fit()

print("\nLogistic Regression Results:")
print(result.summary())
print("\n Odds Ratio Results:")

np.exp(result.params)

params = result.params
conf = result.conf_int()
conf['Odds Ratio'] = params
conf.columns = ['5%', '95%', 'Odds Ratio']
print(np.exp(conf))

### Visualization of Odds Ratio and Regression Coefficients w/ Stimulants

In [ ]:
stimi_odds = {
    'Predictor': [ 'anxiety', 'affective', 'personality'],
    'Odds Ratio': [ 1.850296, 1.977049, 2.137856],
    '5% CI': [ 1.664410, 1.810535, 1.985924],
    '95% CI': [2.056943, 2.158877, 2.301413]
}

stimu_oddsdf = pd.DataFrame(stimi_odds)

plt.figure(figsize=(8, 6))
bars = plt.bar(stimu_oddsdf['Predictor'], stimu_oddsdf['Odds Ratio'], color=['blue', 'hotpink', 'purple', ], edgecolor='black', alpha=0.7)

# add CI error bars; yerr = error bar length for datapoint i n y-axis direc.
plt.errorbar(stimu_oddsdf['Predictor'], stimu_oddsdf['Odds Ratio'], 
             yerr=[stimu_oddsdf['Odds Ratio'] - stimu_oddsdf['5% CI'], stimu_oddsdf['95% CI'] - stimu_oddsdf['Odds Ratio']], 
             fmt='o', color='black', capsize=5, alpha=0.9)

plt.axhline(y=1, color='gray', linestyle='--', linewidth=0.8, label='Baseline (OR=1)')
plt.title('Odds Ratios for Stimulants', fontsize=14, weight = 'bold')
plt.ylabel('Odds Ratio', fontsize=12)
plt.xlabel('Predictor', fontsize=12)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)

#adding astericks for sig
for i, row in stimu_oddsdf.iterrows():
    if row['5% CI'] > 1 or row['95% CI'] < 1:  #  if CI does not include 1
        plt.text(i, row['Odds Ratio'], '*', ha='center', va='bottom', fontsize=14, color='black')

plt.legend(loc='upper right', fontsize=10)
plt.tight_layout()
plt.show()

data_stimulant = {
    'Predictors': ['anxiety', 'affective', 'personality'],
    'Coefficients': [0.6153, 0.6816, 0.7598],
    'Lower CI': [0.509, 0.594, 0.686],
    'Upper CI': [0.721, 0.770, 0.834]
}
coef_df_stimulant = pd.DataFrame(data_stimulant)

colors = ['hotpink', 'teal', 'purple']

# add CI error bars
plt.figure(figsize=(8, 6))
bars = plt.bar(coef_df_stimulant['Predictors'], coef_df_stimulant['Coefficients'], color=colors, edgecolor='black', alpha=0.8)
plt.errorbar(
    coef_df_stimulant['Predictors'],
    coef_df_stimulant['Coefficients'],
    yerr=[
        coef_df_stimulant['Coefficients'] - coef_df_stimulant['Lower CI'],
        coef_df_stimulant['Upper CI'] - coef_df_stimulant['Coefficients']
    ],
    fmt='none',
    ecolor='black',
    capsize=5
)

for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height + 0.05,
             '*',
             ha='center', va='bottom', fontsize=14)

plt.title('Logistic Regression Coefficients for Stimulant ', fontsize=14, weight='bold')
plt.xlabel('Predictors', fontsize=12, weight='bold')
plt.ylabel('Coefficients', fontsize=12, weight='bold')
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()

plt.show()




## <u>Separate Regressions per Stimulant Type<u>

 Given that nicotine abuse comprised over 50% of stimulant cases (see Frequency Charts Section), separate logistic regressions were  conducted for each specific stimulant type (nicotine, amphetamine, cocaine) to determine whether the associations found in the overall stimulant model were predominantly driven by nicotine use rather than reflecting patterns common across all stimulant substances


### Cocaine as Response Variable

In [ ]:
 X = df[['anxiety', 'affective', 'personality'] + covariatelist]
X = sm.add_constant(X)
y = df['COCAINE12']
model = sm.Logit(y, X)
result = model.fit()

print("\nLogistic Regression Results:")
print(result.summary())
odds_ratios = np.exp(result.params)
conf_ints = np.exp(result.conf_int())
odds_ratio_summary = pd.DataFrame({
    'Odds Ratio': odds_ratios,
    'CI Lower': conf_ints[0],
    'CI Upper': conf_ints[1]
})
print(odds_ratio_summary)

### Nicotine as Response Variable

In [ ]:
X = df[['anxiety', 'affective', 'personality'] + covariatelist]
X = sm.add_constant(X)
y = df['NICOTINE12']
model = sm.Logit(y, X)
result = model.fit()

print("\nLogistic Regression Results:")
print(result.summary())

print("\nLogistic Regression Results:")
print(result.summary())
odds_ratios = np.exp(result.params)
conf_ints = np.exp(result.conf_int())
odds_ratio_summary = pd.DataFrame({
    'Odds Ratio': odds_ratios,
    'CI Lower': conf_ints[0],
    'CI Upper': conf_ints[1]
})
print(odds_ratio_summary)

### Amphetamines as Response Variable

In [ ]:
X = df[['anxiety', 'affective', 'personality'] + covariatelist]
X = sm.add_constant(X)
y = df['AMPH12']
model = sm.Logit(y, X)
result = model.fit()

print("\nLogistic Regression Results:")
print(result.summary())
odds_ratios = np.exp(result.params)
conf_ints = np.exp(result.conf_int())
odds_ratio_summary = pd.DataFrame({
    'Odds Ratio': odds_ratios,
    'CI Lower': conf_ints[0],
    'CI Upper': conf_ints[1]
})
print(odds_ratio_summary)

### Visualizing Comparison of Individual and Aggregate Stimulant Abuse Effects

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

results_dict = {
    'Nicotine': {
        'predictors': ['anxiety', 'affective', 'personality'],
        'odds_ratios': [1.86147, 1.95579, 2.12235],
        'ci_lower': [1.67355, 1.78983, 1.97187],
        'ci_upper': [2.06893, 2.13584, 2.28674],
        'significant': [True, True, True]
    },
    'Cocaine': {
        'predictors': ['anxiety', 'affective', 'personality'],
        'odds_ratios': [1.53935, 2.31671, 3.07386],
        'ci_lower': [0.85392, 1.43043, 1.95668],
        'ci_upper': [2.75547, 3.75211, 4.82892],
        'significant': [False, True, True]
    },
    'Amphetamine': {
        'predictors': ['anxiety', 'affective', 'personality'],
        'odds_ratios': [0.92010, 2.27738, 7.52127],
        'ci_lower': [0.45135, 1.27512, 4.07273],
        'ci_upper': [1.87566, 4.06743, 13.88984],
        'significant': [False, True, True]
    },
    'Stimulant': {
        'predictors': ['anxiety', 'affective', 'personality'],
        'odds_ratios': [1.85029, 1.97704, 2.13785],
        'ci_lower': [1.66441, 1.81053, 1.98592],
        'ci_upper': [2.05694, 2.15887, 2.30141],
        'significant': [True, True, True]
    }
}

def forestplot(results_dict):
    fig, ax = plt.subplots(figsize=(10, 8))
    
    colors = {
        'Nicotine': '#16a34a',
        'Cocaine': '#2563eb',
        'Amphetamine': '#dc2626',
        'Stimulant': '#9333ea'
    }
    
    substances = list(results_dict.keys()) #list of substances extracted from key-value pairs in result_sdict
    predictors = results_dict[substances[0]]['predictors'] #for each substance, extracting predictors 
    y_positions = np.arange(len(substances) * len(predictors)) #array of each subst-predictor combo. 
    
    for substance in substances:
        ax.plot([], [], 'o', color=colors[substance], label=substance, markersize=8)
    
    for i, substance in enumerate(reversed(substances)): #REVERsed list to have order of points match order of legend 
        data = results_dict[substance]
        
        for j, predictor in enumerate(predictors):
            y_pos = y_positions[i * len(predictors) + j]
            
            # points w  open circle for non- significance
            if data['significant'][j]:
                ax.plot(data['odds_ratios'][j], y_pos, 'o', color=colors[substance],
                       markersize=8)
            else:
                ax.plot(data['odds_ratios'][j], y_pos, 'o', color=colors[substance],
                       markersize=8, mfc='white', mew=2) #dont fill if nonsig
            
            ax.hlines(y_pos, data['ci_lower'][j], data['ci_upper'][j],
                     color=colors[substance], linewidth=2)
    
    # add vertical line at OR = 1
    ax.axvline(x=1, color='gray', linestyle='--', alpha=0.5)
    
    ax.set_yticks(y_positions)
    y_labels = [pred[:3].capitalize() for sub in reversed(substances) for pred in predictors]#take first three characters of predictor name and capitalize to reduce clutter in graph 
    ax.set_yticklabels(y_labels, fontsize=10, weight='bold')
    
    ax.set_xlabel('Odds Ratio (95% CI)', fontsize=12, weight='bold')
    ax.set_title('Separate and Aggregate Effects of Stimulant Use \n Per Non-SUD Psychiatric Disorder',
                fontweight='bold', fontsize=16, pad=20)
    
    ax.legend(title='Substance Type', loc='upper right', fontsize=10,
             title_fontsize=12)
    
    ax.set_xscale('log')
    ax.set_xlim(0.4, 20)
    
    ax.set_xticks([0.5, 1, 1.5, 2, 3, 5, 10, 15])
    ax.set_xticklabels(['0.5', '1.0', '1.5', '2.0', '3.0', '5.0', '10.0', '15.0'],
                      weight='bold')
    
    # adding backgrond grid
    ax.grid(True, axis='x', alpha=0.3, which='both')
    ax.yaxis.grid(True, linestyle='--', alpha=0.3)
    
    plt.tight_layout()
    
    return fig

fig = forestplot(results_dict)
plt.show()

## <u>Analgesics<u>

In [ ]:
X = df[['anxiety', 'affective', 'personality'] + covariatelist]
X = sm.add_constant(X)
y = df['analgesic']
model = sm.Logit(y, X)
result = model.fit()

print("\nLogistic Regression Results:")
print(result.summary())
print("\n Odds Ratio Results:")

np.exp(result.params)

params = result.params
conf = result.conf_int()
conf['Odds Ratio'] = params
conf.columns = ['5%', '95%', 'Odds Ratio']
print(np.exp(conf))


### Visualization of Odds Ratio and Logistic Regression Coefficients w/ Analgesics

In [ ]:
analg_odddata = {
    'Predictor': ['anxiety', 'affective', 'personality'],
    'Odds Ratio': [1.381205, 2.210541, 2.810784],
    '5% CI': [1.062175, 1.813357, 2.353535],
    '95% CI': [1.796058, 2.694720, 3.356868]
}

analg_odds = pd.DataFrame(analg_odddata)

plt.figure(figsize=(8, 4))
bars = plt.bar(analg_odds['Predictor'], analg_odds['Odds Ratio'], color=['blue', 'hotpink', 'purple'], edgecolor='black', alpha=0.7)

# add CI error bars
plt.errorbar(analg_odds['Predictor'], analg_odds['Odds Ratio'], 
             yerr=[analg_odds['Odds Ratio'] - analg_odds['5% CI'], analg_odds['95% CI'] - analg_odds['Odds Ratio']], 
             fmt='o', color='black', capsize=5, alpha=0.9)

plt.axhline(y=1, color='gray', linestyle='--', linewidth=0.8, label='Baseline (OR=1)')
plt.title('Odds Ratios for Analgesics', fontsize=14, weight = 'bold')
plt.ylabel('Odds Ratio', fontsize=12)
plt.xlabel('Predictor', fontsize=12)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)

#adding astericks for sig
for i, row in analg_odds.iterrows():
    if row['5% CI'] > 1 or row['95% CI'] < 1:  #  if CI does not include 1
        plt.text(i, row['Odds Ratio'], '*', ha='center', va='bottom', fontsize=14, color='black')

plt.legend(loc='upper right', fontsize=10)
plt.tight_layout()
plt.show()

dataanalgesic = {
    'Predictors': ['anxiety', 'affective', 'personality'],
    'Coefficients': [0.3230, 0.7932, 1.0335],
    'Lower CI': [0.060, 0.595, 0.856],
    'Upper CI': [0.586, 0.991, 1.211]
}
df_analg = pd.DataFrame(dataanalgesic)

colors = ['hotpink', 'teal', 'purple']
plt.figure(figsize=(8, 6))
bars = plt.bar(df_analg['Predictors'], df_analg['Coefficients'], color=colors, edgecolor='black', alpha=0.8)
plt.errorbar(
    df_analg['Predictors'],
    df_analg['Coefficients'],
    yerr=[
        df_analg['Coefficients'] - df_analg['Lower CI'],
        df_analg['Upper CI'] - df_analg['Coefficients']
    ],
    fmt='none',
    ecolor='black',
    capsize=5
)
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height + 0.05,
             '*',
             ha='center', va='bottom', fontsize=14)


plt.title('Logistic Regression Coefficients for Analgesics ', fontsize=14, weight='bold')
plt.xlabel('Predictors', fontsize=12, weight='bold')
plt.ylabel('Coefficients', fontsize=12, weight='bold')
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()

plt.show()


# Multinomial Regression: Comparing No, Single, and Poly-Substance Abuse

In [ ]:
# filtering for mutli drug usage 
def drugtypecreate(row):
    if row['stimulant'] == 1 and row['depressant'] == 0 and row['analgesic'] == 0:
        return 'stimulant'
    elif row['stimulant'] == 0 and row['depressant'] == 1 and row['analgesic'] == 0:
        return 'depressant'
    elif row['stimulant'] == 0 and row['depressant'] == 0 and row['analgesic'] == 1:
        return 'analgesic'
    elif row['stimulant'] == 0 and row['depressant'] == 0 and row['analgesic'] == 0:
        return '_aaaanone' 
    else:
        return 'poly'  # polydrug use aka more than just one category

# new variable for type of drug
df['drugtype'] = df.apply(drugtypecreate, axis=1)

print("\nDrug Type Distribution:")
print(df['drugtype'].value_counts())
print("\nDrug Type Distribution (Percentages):")
print(df['drugtype'].value_counts(normalize=True) * 100)


In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.discrete.discrete_model import MNLogit
X = df[['anxiety', 'affective', 'personality']]
X = sm.add_constant(X)

#named _aaanone so that 'none' category is alphabetically first (and therefore the reference catego)
y = pd.Categorical(df['drugtype'],categories=['_aaaanone', 'stimulant', 'depressant', 'poly', 'analgesic'])
print (y.categories.sort_values())
model = MNLogit(y, X)
result = model.fit()
print (result.summary())
odds_ratios = np.exp(result.params)

#num of categories excluding reference 
n_categories = len(y.categories) - 1

# converting array of odds ratio to matrix
odds_ratios_matrix = odds_ratios.values.reshape(4, n_categories)

odds_ratios_df = pd.DataFrame(
    odds_ratios_matrix,
    index=['const', 'anxiety', 'affective', 'personality'],
    columns=[cat for cat in y.categories if cat != y.categories[0]]
)

print("\nReference category:", y.categories[0])
print("\nOdds Ratios (compared to reference category):")
print(odds_ratios_df)



print("\nOdds Ratios:")
print(np.exp(result.params))




## Visualization of Multinomial Regression Coefficients

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

drug_types = ['stimulant', 'depressant', 'poly', 'analgesic']
predictors = ['anxiety', 'affective', 'personality']

#  arrays for coefficients and standard errors
coeffs = {
    'anxiety': [0.6350, -0.0534, 0.2964, -0.0091],
    'affective': [0.7538, 0.6662, 1.2160, 0.8915],
    'personality': [0.7791, 0.5774, 1.3367, 1.1036]
}

std_errs = {
    'anxiety': [0.059, 0.097, 0.089, 0.290],
    'affective': [0.050, 0.069, 0.069, 0.209],
    'personality': [0.042, 0.058, 0.062, 0.177]
}

# P-values from the regression output
p_values = {
    'anxiety': [0.000, 0.580, 0.001, 0.975],
    'affective': [0.000, 0.000, 0.000, 0.000],
    'personality': [0.000, 0.000, 0.000, 0.000]
}

fig, ax = plt.subplots(figsize=(10, 6))

width = 0.25
x = np.arange(len(drug_types))

bars = []
bars.append(ax.bar(x - width, coeffs['anxiety'], width, label='Anxiety', 
                  yerr=std_errs['anxiety'], capsize=5, color='#87CEEB'))
bars.append(ax.bar(x, coeffs['affective'], width, label='Affective', 
                  yerr=std_errs['affective'], capsize=5, color='#90EE90'))
bars.append(ax.bar(x + width, coeffs['personality'], width, label='Personality', 
                  yerr=std_errs['personality'], capsize=5, color= 'purple'))


for i, predictor in enumerate(['anxiety', 'affective', 'personality']):
    for j, p_val in enumerate(p_values[predictor]):
        if p_val < 0.05:
            height = coeffs[predictor][j]
            offset = max(std_errs[predictor][j], 0.1)  
            ax.text(x[j] + (i-1)*width, height + offset, '*', 
                   ha='center', va='bottom')

ax.set_ylabel('Regression Coefficients')

plt.suptitle('Multinomial Regression Coefficients by Drug Type', 
            fontweight='bold', 
            fontsize=14, y=0.95) 


# subtitle 
plt.title('Reference Category: No Drug Use \n*asterisk indicates p<0.05', 
         fontsize=10, 
         fontweight='normal', x= 0.45,y=0.85,
         pad=15)
ax.set_xticks(x)
ax.set_xticklabels(drug_types)
ax.legend()

ax.axhline(y=0, color='grey', linestyle='--', alpha=0.3)

ax.grid(True, axis='y', linestyle='--', alpha=0.3)

plt.xticks(rotation=0)

plt.tight_layout()

plt.show()

Multinomial odds ratio visualization shown in section below

# Aggregate Logistic and Multinomial Odd Ratio Results

## <u>What predicts using each specific substance?<u>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

predictors = ['anxiety', 'affective', 'personality']
substances = ['Stimulants', 'Depressants', 'Analgesics']

odds_ratios = {
    'Stimulants':  [1.85, 2.0, 2.1],    
    'Depressants': [1.3, 1.95, 1.8],  
    'Analgesics':  [1.4, 2.2, 2.8]      
}

ci_errors = {
    'Stimulants':  [(0.2, 0.2), (0.15, 0.15), (0.15, 0.15)],
    'Depressants': [(0.2, 0.2), (0.15, 0.15), (0.15, 0.15)],
    'Analgesics':  [(0.3, 0.3), (0.4, 0.4), (0.4, 0.4)]
}

def groupedbarchart():
    plt.figure(figsize=(12, 6))
    bar_width = 0.25
    r1 = np.arange(len(predictors))
    r2 = [x + bar_width for x in r1]
    r3 = [x + bar_width for x in r2]
    colors = ['#3b82f6', '#ec4899', '#9333ea']
    
    bars1 = plt.bar(r1, odds_ratios['Stimulants'], bar_width, label='Stimulants', color=colors[0], 
            yerr=list(zip(*ci_errors['Stimulants'])), capsize=5)
    bars2 = plt.bar(r2, odds_ratios['Depressants'], bar_width, label='Depressants', color=colors[1], 
            yerr=list(zip(*ci_errors['Depressants'])), capsize=5)
    bars3 = plt.bar(r3, odds_ratios['Analgesics'], bar_width, label='Analgesics', color=colors[2], 
            yerr=list(zip(*ci_errors['Analgesics'])), capsize=5)

    for bars, x_pos in [(bars1, r1), (bars2, r2), (bars3, r3)]:
        for i, bar in enumerate(bars):
            height = bar.get_height()
            err = ci_errors[list(odds_ratios.keys())[list((bars1, bars2, bars3)).index(bars)]][i][1]
            plt.text(x_pos[i], height + err + 0.05, '*', 
                    ha='center', va='bottom')

    plt.axhline(y=1, color='gray', linestyle='--', alpha=0.5, label='Baseline (OR=1)')
    
    plt.xlabel('NSPD Type')
    plt.ylabel('Odds Ratio (95% CI)')
    
    plt.suptitle('Logistic Regression Analyses: Odd Ratios by Substance Class', 
            fontweight='bold', 
            fontsize=14, y=0.95)  # Adjust vertical position of main title

# subtitle 
    plt.title('\n*asterisk indicates p<0.05', fontsize=10, fontweight='normal', x= 0.45,y=0.85,pad=15)
    plt.xticks([r + bar_width for r in range(len(predictors))], predictors)
    plt.legend()
    
    plt.grid(True, axis='y', alpha=0.3)
    
    plt.tight_layout()
    
    return plt.gcf()

fig = groupedbarchart()
plt.show()

## <u>What predicts different patterns of drug use behavior?<u>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter

drug_types = ['stimulant', 'depressant', 'poly', 'analgesic']

# odds for second multinomial using pd.categorical
anxiety_vals = [0.990935, 0.948028 , 1.344967,  1.887005]
affective_vals = [2.438847 ,1.946802 ,3.373768,  2.125127]
personality_vals = [3.014986,1.781420,  3.806444,  2.179440]

x = np.arange(len(drug_types))
width = 0.25  

fig, ax = plt.subplots(figsize=(7, 6))

# BARS for each predic
ax.bar(x - width, anxiety_vals, width, label='anxiety', color='pink', alpha=0.7)
ax.bar(x, affective_vals, width, label='affective', color='red')
ax.bar(x + width, personality_vals, width, label='personality', color='teal')
# Add asterisks for significant results
# Anxiety (only significant for poly and stimulant)
ax.text(x[0] - width, anxiety_vals[0] + 0.1, '*', ha='center', va='bottom')  # stimulant
ax.text(x[2] - width, anxiety_vals[2] + 0.1, '*', ha='center', va='bottom')  # poly

# Affective (significant for all)
for i in range(len(drug_types)):
    ax.text(x[i], affective_vals[i] + 0.1, '*', ha='center', va='bottom')

# Personality (significant for all)
for i in range(len(drug_types)):
    ax.text(x[i] + width, personality_vals[i] + 0.1, '*', ha='center', va='bottom')

#formatting y-axis vals
def format_func(x, p):
    return f"{x:.2f}"
ax.yaxis.set_major_formatter(FuncFormatter(format_func))

ax.set_xlabel('Drug Usage (No Drug Usage as Reference Category)')
ax.set_ylabel('Odds Ratios')
plt.suptitle('Psychiatric Disorder Odd Ratios \n with Drug Use Patterns: Single, Poly, and No Use',
            fontweight='bold', 
            fontsize=14, y=0.95)  # Adjust vertical position of main title


# subtitle 
plt.title('Reference Category: No Drug Use \n*asterisk indicates p<0.05', 
         fontsize=10, 
         fontweight='normal', x= 0.45,y=0.85,
         pad=15)

#formatting y-axis vals
def format_func(x, p):
        return f"{x:.2f}"
ax.yaxis.set_major_formatter(FuncFormatter(format_func))

ax.set_xlabel('Drug Usage (No Drug Usage as Reference Category)')
ax.set_ylabel('Odds Ratios')
ax.set_xticks(x)
ax.set_xticklabels(drug_types)
ax.legend(title='Predictors', bbox_to_anchor=(1.05, 1), loc='upper left')
ax.grid(True, which="both", ls="-", alpha=0.2)

ax.set_ylim(0, 5)  #  all values are between 0 and 2

ax.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()

plt.show()